# Assignment: Segmenting and Clustering Neighborhoods in Toronto

# Problem Statement

## Brief
>1. Acquire the data, 
2. Explore the data, 
3. Segment, and Cluster the neighborhoods in the city of Toronto based on the postalcode and borough information.

### 1. Part I - Data Acquisition

>The Toronto neighborhood data can be found on a Wikipedia page that has all the information we need to explore and cluster the neighborhoods in Toronto. We will have to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format.

Create a dataframe consisting of three columns: PostalCode, Borough, and Neighborhood from `'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.'`
- Only process the cells that have an assigned borough. 
- Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. Combine multiple rows into single rows with comma-separated `'Neighborhoods'`.
- If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.



### 1.1 Method

#### Import the required Libraries

In [1]:
!pip install folium
import folium
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # this module helps us to download a web page
from bs4 import BeautifulSoup # this module helps in web scrapping.

print('Libraries imported.')

Libraries imported.


#### 1.1.1 Get Data from Wiki.
Define URL and get via requests

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #Wiki url
req = requests.get(URL)

#### 1.1.2 Pass to BeautifulSoup.
First, the document is converted to Unicode, (similar to ASCII), 
and HTML entities are converted to Unicode characters. 
Beautiful Soup transforms a HTML document into a tree of Python objects.

In [3]:
soup = BeautifulSoup(req.content, 'html5lib') 
print('Page Scraped.') 

Page Scraped.


#### 1.1.3 Create a dataframe from the scraped data
>1. Create a list to store the scraped data
2. Find the 1st table in the scraped data represented by the tag \<table\>
3. Find all tags \<tr\> in the table
4. Create a dictionary to store data
5. Ignore Not assigned values.
6. Postal code contains up to 3 characters from the \<p\> tag
7. Split Borough from the \<span\> tag
8. Split Neighborhood from the \<span\> tag
9. Append to list
10. Change list to dataframe
11. Standardize Borough
12. Reset the index
13. Combine multiple rows into single rows (multiple neighborhoods can exist in one postal code area)
14. Check the health of the dataframe


In [4]:
table_contents=[] # Create a list to store the scraped data
table=soup.find('table') # Find the 1st table in the scraped data represented by the tag <table>
for row in table.findAll('td'): # Find all tags <tr> in the table
    cell = {} # create a dictionary to store data
    if row.span.text=='Not assigned':
        pass # Ignore Not assigned.
    else:
        cell['PostalCode'] = row.p.text[:3] # postal code contains up to 3 characters from the <p> tag
        cell['Borough'] = (row.span.text).split('(')[0] # Split Borough from the <span> tag
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ') # Split Neighborhood from the <span> tag
        table_contents.append(cell) # Append to list

df=pd.DataFrame(table_contents) # Change list to dataframe

df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'}) # Standardize
df.reset_index(inplace = True) # Reset the index
df=df.groupby(['PostalCode']).first() # More than one neighborhood can exist in one postal code area. Combine multiple rows into single rows with comma-separated 'Neighborhoods'
df.head()

,index,Borough,Neighborhood
PostalCode,,,
M1B,6,Scarborough,"Malvern, Rouge"
M1C,12,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,18,Scarborough,"Guildwood, Morningside, West Hill"
M1G,22,Scarborough,Woburn
M1H,26,Scarborough,Cedarbrae


In [5]:
print('Number of unique Postal Codes is {}'.format(len(df['Neighborhood'].unique())))

Number of unique Postal Codes is 103


In [6]:
print('Number of Borough\'s \'Not assigned\' is {}'.format(df[df['Borough'] == 'Not assigned'].count()[0]))

Number of Borough's 'Not assigned' is 0


In [7]:
df.shape

(103, 3)

### 2. Part II - Data Exploration

> The Toronto neighbourhood data from Wiki does not contain geospatial data. We need to get the latitude and the longitude coordinates of each `postal code` merged with this data in order to use FourSquare the API for additional feature-data. 
We will link to a csv (https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv) file that has the geographical coordinates of each postal code.


### 2.1 Method

#### Import the required Libraries

In [8]:
!pip install folium
import folium
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # this module helps us to download a web page
from bs4 import BeautifulSoup # this module helps in web scrapping.

print('Libraries imported.')

Libraries imported.Requirement already satisfied: idna<3,>=2.5 in c:\users\conta\anaconda3\lib\site-packages (from requests->folium) (2.10)



#### 2.1.1 Define URL to csv-file

In [9]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'

#### 2.1.2 Read GeoSpatial Data from csv-file

In [10]:
df_geo = pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### 2.1.3 Rename column `PostalCode` 

In [11]:
df_geo.rename(columns = {'Postal Code':'PostalCode'}, inplace = True) # Rename column
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### 2.1.4 Confirm DataTypes of DataFrames

In [12]:
df.dtypes,df_geo.dtypes # DataTypes of DataFrames

(index            int64
 Borough         object
 Neighborhood    object
 dtype: object,
 PostalCode     object
 Latitude      float64
 Longitude     float64
 dtype: object)

#### 2.1.5 Confirm Shapes of DataFrames

In [13]:
df.shape,df_geo.shape # Shapes of DataFrames

((103, 3), (103, 3))

#### 2.1.6 Join the dataframes (neighborhood postal code & geographical coordinates) 

In [14]:
df_joined = df.join(df_geo.set_index('PostalCode'), on='PostalCode', how='inner')
df_joined

,index,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,,
M1B,6,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,12,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,18,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,22,Scarborough,Woburn,43.770992,-79.216917
M1H,26,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
M9N,64,York,Weston,43.706876,-79.518188
M9P,70,Etobicoke,Westmount,43.696319,-79.532242
M9R,77,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724


### 3. Part III - Data Processing

>Explore and cluster the neighborhoods in Toronto. We cluster Toronto based on the similarities of the venues categories using Kmeans clustering.

### 3.1 Method

#### Import the required Libraries

In [15]:
from geopy.geocoders import Nominatim 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#### 3.1.1 Get the latitude, longitude coordinates of Toronto.

In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


##### 3.1.2 Visualize the map of Toronto with neighbourhoods superimposed on top.
>1. Create the map of Toronto
2. Add markers to map
3. Display Map

In [17]:
# Creating the map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for latitude, longitude, borough, neighbourhood in zip(df_joined['Latitude'], df_joined['Longitude'], df_joined['Borough'], df_joined['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True
        ).add_to(map_Toronto)  
    
map_Toronto

#### 3.1.3 Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'IP0OZLOE2KUMJU4MC3GIYEKAQG43JD1HBTVT1FT4QTESG05P' # your Foursquare ID
CLIENT_SECRET = 'KLUUZDERL5RAUSJKN1Q3DFUQ4T2FPSUQNSJR1MYMO15XJ2LB' # your Foursquare Secret
ACCESS_TOKEN = 'CLP1Z4FAXZEGSJ1NJOPJMXA1NHHBLBOHVQGBZMHTLHXKIS30' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IP0OZLOE2KUMJU4MC3GIYEKAQG43JD1HBTVT1FT4QTESG05P
CLIENT_SECRET:KLUUZDERL5RAUSJKN1Q3DFUQ4T2FPSUQNSJR1MYMO15XJ2LB


#### 3.1.4 Function to get all the nearby venues for neighbourhoods for `'Toronto, Ontario'`

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #results = requests.get(url).json()
        #print(results)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #print(results)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### 3.1 5 Create a new dataframe called toronto_venues.

In [20]:
toronto_venues = getNearbyVenues(names=df_joined['Neighborhood'],
                                   latitudes=df_joined['Latitude'],
                                   longitudes=df_joined['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale


#### 3.1.6 Shape of toronto_venues dataframe

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(2112, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


#### 3.1.7 Count of venues for each neighborhood

In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
...,...,...,...,...,...,...
Willowdale West,6,6,6,6,6,6
"Willowdale, Newtonbrook",1,1,1,1,1,1
Woburn,3,3,3,3,3,3


#### 3.1.8 Number of unique categories curated from all the returned venues.

In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


#### 3.1.9 Analyze Each Neighborhood

#### 3.1.9.1 'one hot encoding' for toronto_venues-dataframe

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 3.1.9.2 New dataframe size

In [25]:
toronto_onehot.shape

(2112, 271)

#### 3.1.9.3 Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
96,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0


#### 3.1.9.4 Grouped dataframe size¶

In [27]:
toronto_grouped.shape

(100, 271)

#### 3.1.10.1 Print each neighborhood along with the top 5 most common venues

In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4         Mexican Restaurant  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1        Pharmacy  0.11
2            Pool  0.11
3  Sandwich Place  0.11
4             Pub  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0                 Bank  0.09
1          Coffee Shop  0.09
2          Bridal Shop  0.04
3            Gift Shop  0.04
4  Fried Chicken Joint  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4        Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop  0.08
1               Restaurant  0.08
2           Sandwich

4        Coffee Shop   0.2


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                           Café  0.08
1                    Coffee Shop  0.07
2  Vegetarian / Vegan Restaurant  0.05
3          Vietnamese Restaurant  0.05
4            Arts & Crafts Store  0.03


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                             venue  freq
0                   Sandwich Place   0.5
1                             Park   0.5
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0          Clothing Store  0.33
1  Furniture / Home Store  0.17
2           Women's Store  0.08
3   Vietnamese Restaurant  0.08
4      Miscellaneous Shop  0.08


----Lawrence Park----
                       venue  freq
0                   Bus Line  0.33
1                       Park  

           venue  freq
0       Pharmacy  0.17
1  Grocery Store  0.17
2    Supermarket  0.17
3    Coffee Shop  0.17
4        Butcher  0.17


----Willowdale, Newtonbrook----
                             venue  freq
0                             Park   1.0
1                      Yoga Studio   0.0
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Woburn----
                   venue  freq
0            Coffee Shop  0.67
1  Korean BBQ Restaurant  0.33
2            Yoga Studio  0.00
3     Miscellaneous Shop  0.00
4    Moroccan Restaurant  0.00


----Woodbine Heights----
          venue  freq
0  Skating Rink  0.11
1           Spa  0.11
2          Park  0.11
3  Intersection  0.11
4   Video Store  0.11


----York Mills West----
                 venue  freq
0    Electronics Store  0.33
1                 Park  0.33
2    Convenience Store  0.33
3          Yoga Studio  0.00
4  Monument / Landmark  0.00




#### 3.1.10.2 Function to sort the venues in descending order

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### 3.1.10.3 New dataframe and display the top 10 venues for each neighborhood

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Playground,Pool,Pub,Gym,Airport Service,Falafel Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Sushi Restaurant,Shopping Mall,Bridal Shop,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Park
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Sushi Restaurant,Pharmacy,Pizza Place,Pub,Café,Butcher


#### 3.1.11.1 Cluster Neighborhoods - Run k-means to cluster the neighborhood into 5 clusters

In [31]:
from sklearn.cluster import KMeans

In [32]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 4, 0, 0, 0, 0, 0, 1, 0, 0, 1])

#### 3.1.11.2 New dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_joined

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(50) # check the last columns!

,index,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,,,,,
M1B,6,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Dim Sum Restaurant
M1C,12,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Bar,Moving Target,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dim Sum Restaurant
M1E,18,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Donut Shop,Mexican Restaurant,Intersection,Electronics Store,Restaurant,Rental Car Location,Breakfast Spot,Medical Center,Bank,Distribution Center
M1G,22,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
M1H,26,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Gas Station,Athletics & Sports,Bakery,Caribbean Restaurant,Thai Restaurant,Bank,Fried Chicken Joint,Drugstore,Donut Shop
M1J,32,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Pizza Place,Playground,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
M1K,38,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,0.0,Department Store,Discount Store,Coffee Shop,Hobby Shop,Convenience Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
M1L,44,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,0.0,Bus Line,Bakery,Ice Cream Shop,Intersection,Park,Metro Station,Bus Station,Soccer Field,Creperie,Convenience Store
M1M,51,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0.0,Motel,American Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop


#### 3.1.11.3 Drop any Nan values to prevent data interference

In [34]:
initialShape = toronto_merged.shape
if toronto_merged.isnull().values.any():
    toronto_merged = toronto_merged.dropna(subset=['Cluster Labels'])
print('Shape change from {} to {} due to NaN'.format(initialShape, toronto_merged.shape))

Shape change from (103, 16) to (100, 16) due to NaN


#### 3.1.11.4 Visualize the resulting clusters

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 3.1.11.4 Confirm Cluster # 1

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,index,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,
M1C,12,0.0,Bar,Moving Target,Women's Store,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dim Sum Restaurant
M1E,18,0.0,Donut Shop,Mexican Restaurant,Intersection,Electronics Store,Restaurant,Rental Car Location,Breakfast Spot,Medical Center,Bank,Distribution Center
M1G,22,0.0,Coffee Shop,Korean BBQ Restaurant,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
M1H,26,0.0,Hakka Restaurant,Gas Station,Athletics & Sports,Bakery,Caribbean Restaurant,Thai Restaurant,Bank,Fried Chicken Joint,Drugstore,Donut Shop
M1J,32,0.0,Pizza Place,Playground,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
M9L,50,0.0,Pizza Place,Intersection,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Farmers Market
M9N,64,0.0,Convenience Store,Jewelry Store,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
M9P,70,0.0,Intersection,Pizza Place,Discount Store,Sandwich Place,Coffee Shop,Middle Eastern Restaurant,Chinese Restaurant,Women's Store,Dog Run,Distribution Center


#### 3.1.11.4 Confirm Cluster # 2

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,index,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,
M1V,85,1.0,Park,Playground,Intersection,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
M2M,52,1.0,Park,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
M2P,66,1.0,Park,Electronics Store,Convenience Store,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
M3A,0,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
M3K,40,1.0,Park,Airport,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
M4N,61,1.0,Park,Bus Line,Swim School,Women's Store,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
M4W,91,1.0,Park,Playground,Trail,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
M6E,21,1.0,Park,Women's Store,Pool,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
M8X,98,1.0,Park,River,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore


#### 3.1.11.4 Confirm Cluster # 3

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,index,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,
M1B,6,2.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Dim Sum Restaurant


#### 3.1.11.4 Confirm Cluster # 4

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,index,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,
M3M,53,3.0,Baseball Field,Food Truck,Women's Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Diner
M8Y,101,3.0,Baseball Field,Breakfast Spot,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store
M9M,57,3.0,Baseball Field,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store,Fast Food Restaurant


#### 3.1.11.4 Confirm Cluster # 5

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,index,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,
M9B,11,4.0,Bakery,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


# Toronto neighbourhood Clustering based on venue categories - Done